In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/Users/tareen/Desktop/Research_Projects/2022_mavenn2_github/mavenn')

import mavenn
import logomaker
import seaborn as sns
import re

%matplotlib inline

In [2]:
# Load example data
data_df = mavenn.load_example_dataset('sortseq')

# Separate test from data_df
ix_test = data_df['set']=='test'
test_df = data_df[ix_test].reset_index(drop=True)
print(f'test N: {len(test_df):,}')

# Remove test data from data_df
data_df = data_df[~ix_test].reset_index(drop=True)
print(f'training + validation N: {len(data_df):,}')
data_df.head()

test N: 9,935
training + validation N: 40,583


set  ct_0  ct_1  ct_2  ct_3  ct_4  ct_5  ct_6  ct_7  ct_8  ct_9  \
0    training     0     1     0     0     0     0     0     0     0     0   
1    training     0     0     0     0     0     0     0     0     0     1   
2    training     0     0     0     0     0     0     0     0     0     1   
3    training     0     1     0     0     0     0     0     0     0     0   
4  validation     0     0     0     0     0     0     1     0     0     0   

                                                   x  
0  AAAAAAAGTGAGTTAGCCAACTAATTAGGCACCGTACGCTTTATAG...  
1  AAAAAATGAGAGTTAGTTCACTCATTCGGCACCACAGGCTTTACAA...  
2  AAAAAATGGGTGTTAGCTCTATCATTAGGCACCCCCGGCTTTACAC...  
3  AAAAAATGTCAGTTAGCTGACTCATTAGGCACCCCTGGCTTTACGT...  
4  AAAAAATGTGAGAAAGCTCACTCCTTTGGCACCGCAGGCTTTACAC...

In [3]:
# load weights and set these weights to current additive gpmap to 
# check whether x_to_phi is working correctly
theta = pd.read_csv('data/sortseq_mpa_additive_theta.csv',index_col=[0])
theta.head()


A         C         G         T
0 -0.034530 -0.041985 -0.006270  0.164501
1  0.054123 -0.174570  0.011680 -0.104753
2 -0.461191  0.086680  0.078193 -0.877013
3 -1.345192 -1.477536 -1.236911  0.145073
4 -0.574200 -1.580839 -0.523247  0.086954

In [4]:
# set positional argumnets for gpmap function
L = len(data_df['x'][0])
alphabet=['A','C','G','T']

# get test set data.
x_test = test_df['x'].values

In [5]:
# instantiate gpmap object
gpmap = mavenn.gpmap.AdditiveGPMapLayer(L, alphabet)

# this isn't working, need to fix
# gpmap.set_params(theta_0=-1.1306967676897681,
#                  theta_lc=theta.values)



2022-03-22 13:52:58.355885: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Updated phi usage
Computed directly from gpmap object

In [6]:
phi = gpmap.x_to_phi(x_test)
phi

array([1.9787908 , 1.6849082 , 0.79621434, ..., 1.8998104 , 2.1030018 ,
       1.7040522 ], dtype=float32)

### Create measurement process object(s)

In [7]:
# I think this shouldn't require any positional arguments.
# mp_GE = mavenn.measurement_process_layers.GlobalEpistasisLayer(K=50,
#                                                                eta=1e-5,
#                                                                monotonic=True)
mp_mpa = mavenn.measurement_process_layers.DiscreteAgnosticMP(info_for_layers_dict={'H_y_norm':0},
                                                                      Y=10,
                                                                      K=50,
                                                                      eta=1e-5,)


about to call super from discrete Agnostic


In [8]:
model = mavenn.Model2(gpmap=gpmap,
                      mp_list=[mp_mpa])

Building up output layers
 Number input nodes: 300, number outputs = 10


In [9]:
model.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Sequence_labels_input (InputLa  [(None, 310)]       0           []                               
 yer)                                                                                             
                                                                                                  
 Sequence_only (Lambda)         (None, 300)          0           ['Sequence_labels_input[0][0]']  
                                                                                                  
 additive_gp_map_layer (Additiv  (None, 1)           301         ['Sequence_only[0][0]']          
 eGPMapLayer)                                                                                     
                                                                                              